# Pricing research — Van Westendorp’s Price Sensitivity Meter in Python
*Survey design and analysis in Python*

## Survey
Van Westendorp’s price sensitivity meter is designed to investigate customer price preferences. The price sensitivity meter survey consists of four questions asking each survey participants to provide four price points of the product when the product is:
- Too cheap (i.e., so inexpensive)
- Cheap (i.e., a bargain)
- Expensive
- Too expensive
An example of the four questions shows as follows:

![](price_sensitivity.png)
*Source: Qualtrics*

## Data
Assuming 20 participants have filled in the survey. Let’s first import needed libraries and read in data from these 20 participants.

In [ ]:

import pandas as pd
import hvplot.pandas 
import holoviews as hv

prices = {'Too Cheap': [100,120,200,200,300,100,100,300,100,350,340,450,100,257,109,109,280,400,250,200],
          'Cheap': [150,200,250,300,340,190,200,350,120,360,360,460,110,388,299,129,350,410,260,240],
          'Expensive': [400,400,450,350,400,200,300,370,180,370,490,490,130,433,399,149,400,420,270,280],
          'Too Expensive': [500,480,500,400,490,300,500,380,200,380,500,500,140,499,422,199,410,430,280,300],
        }

df = pd.DataFrame(prices)

The first five rows of the data frame look like this:

In [ ]:
df.head()

## Analysis
Here I wrote a function to calculate the cumulative percentage of the four questions (Too Cheap, Cheap, Expensive, Too Expensive), plot the four lines, and report on the optimal price and acceptable price range.

In [ ]:
def price_sensitivity_meter(df, interpolate=False):
    # convert data from wide to long
    # calculate frequency of each price for each group
    df1 = (df[['Too Cheap', 'Cheap', 'Expensive', 'Too Expensive']]
             .unstack()
             .reset_index()
             .rename(columns = {'level_0':'label', 0: 'prices'})[['label','prices']]
             .groupby(['label','prices'])
             .size()
             .reset_index()
             .rename(columns = {0: 'frequency'})
            )
    # calculate cumsum percentages
    df1['cumsum'] = df1.groupby(['label'])['frequency'].cumsum()
    df1['sum'] = df1.groupby(['label'])['frequency'].transform('sum')
    df1['percentage'] = 100*df1['cumsum']/df1['sum']
    # convert data from long back to wide
    df2 = df1.pivot_table('percentage', 'prices', 'label')
    
    # take linear values in missing values
    if interpolate:
        df3 = df2.interpolate().fillna(0)
        df3['Too Cheap'] = 100 - df3['Too Cheap']
        df3['Cheap'] = 100 - df3['Cheap']
        plot = df3.hvplot(x='prices', 
                          y=['Too Cheap', 'Cheap', 'Expensive', 'Too Expensive'],
                          ylabel = 'Percentage',
                          height=400,
                          color=['green','lightgreen','lightpink','crimson']
                              ).opts(legend_position='bottom')
    
    # forward fill 
    else: 
        df3 = df2.ffill().fillna(0)
        
        df3['Too Cheap'] = 100 - df3['Too Cheap']
        df3['Cheap'] = 100 - df3['Cheap']
        plot = df3.hvplot.step(x='prices', 
                               y=['Too Cheap', 'Cheap', 'Expensive', 'Too Expensive'],
                               where='post',
                               ylabel = 'Percentage',
                               height=400,
                               color=['green','lightgreen','lightpink','crimson']
                              ).opts(legend_position='bottom')
    df3['optimal_diff'] = (df3['Too Cheap'] - df3['Too Expensive'])
    df3['left_diff'] = (df3['Too Cheap'] - df3['Expensive'])
    df3['right_diff'] = (df3['Too Expensive'] - df3['Cheap'])
    optimal = df3[df3['optimal_diff']<=0].index[0]
    lower_bound = df3[df3['left_diff']<=0].index[0]
    upper_bound = df3[df3['right_diff']>=0].index[0]
    

    optimal_line = hv.VLine(optimal).opts(color='blue', line_dash='dashed', line_width=0.4)

    lower_line = hv.VLine(lower_bound).opts(color='grey', line_dash='dashed', line_width=0.4)
    upper_line = hv.VLine(upper_bound).opts(color='grey', line_dash='dashed', line_width=0.4)

    
    print(f'Optimal Price: ${optimal}')
    print(f'Acceptable Price Range: ${lower_bound} to ${upper_bound}')
    
    return plot * lower_line * optimal_line * upper_line

Here is the output when we run `price_sensitivity_meter(df)`. The x-axis shows the price points, the y-axis shows the percentage of the cumulative frequencies of the prices. If you would like the plot to be more smooth, you can set `interpolate=True`. The default value of interpolate is False in our function.

In [ ]:
price_sensitivity_meter(df)

## The key results are:
- Optimal price point (OPP)

The intersection of “Too Cheap” and “Too Expensive”. In our example, the optimal price is \$300.

- Acceptable price range

The lower bound (aka. point of marginal cheapness or PMC) is the intersection of “Too Cheap” and “Expensive”. The upper bound (aka. point of marginal expensiveness or PME.) is the intersection of “Cheap” and “Too Expensive”. In our example, the acceptable price range is \$280 to \$350.

## Validating results with Qualtrics
To make sure our results are correct, I validated my results with Qualtrics. With the same data, Qualtrics produces the following report:
![](price_sensitivity2.png)

The optimal price is the same as our result. Both show \\$300 as the optimal price. The acceptable price range is different but very close. We got \$280 to \$350, while Qualtrics reported \$280.1 to \$350.0. (I am not sure why Qualtrics has a slope in each step in the plot. The vertical lines between steps are not straight. I suspect that with large data size, the vertical lines will become straight.)

Now you can design your own Price Sensitivity Meter survey and analyze results in Python. Enjoy!


By Sophia Yang on [October 2, 2020](https://towardsdatascience.com/pricing-research-van-westendorps-price-sensitivity-meter-in-python-ec07fabbeacd).